# Info

La información sobre los porque y contexto del proyecto así como su información acerca de las variables fue guardada en Github (https://github.com/devFrow/Proyecto_trancitorio.git)
para una mayor comodidad a la hora de trabajar con el código. 


### Comienzo 

El tratado de duplicados y nulos se hace en el archivo Tratamiento_nulos.ipynb que se encuentra dentro del mismo repositorio. 
Aquí vamos a comenzar directamente con el EDA. 

| **Alrededores y Forma** | **Ubicación**  | **Tamaños** | **Estilos** | **Estados** | **Tiempo** | **Techo** | **Revestimiento** | **Porch** | **Venta** |
|---------------------|------------|---------|---------|---------|--------|-------|---------------|-------|-------|
|Street|MSZoning|LotFrontage|MSSubClass|OverallQual|YearBuilt|	RoofMatl|	Exterior1st|	OpenPorchSF|	SaleType|
|Alley|	Neighborhood|	LotArea	|BldgType|	OverallCond|	YearRemodAdd|	RoofStyle|	Exterior2nd|	EnclosedPorch|	SaleCondition|
LotShape|	Condition1|	TotalBsmtSF|HouseStyle|Functional|	MoSold|		|MasVnrType|	3SsnPorch|	|
LandContour|	Condition2|	1stFlrSF|	|	|YrSold	|	|MasVnrArea|	ScreenPorch	||
LotConfig|	|	2ndFlrSF|		|	|||		ExterQual|||
LandSlope||		LowQualFinSF|||||					ExterCond|||		
|||GrLivArea||||||||	

\n

|**Cimientos**|**Sótano**|**Calefacción/Aire**|**Servicios**|**Baños**|**Habitaciones**|**Cocina**|**Garage**|**Extras**|
|-------------|----------|--------------------|-------------|---------|----------------|----------|----------|----------|
Foundation|BsmtQual|Heating|Utilities|BsmtFullBath|Bedroom|Kitchen|GarageType|WoodDeckSF|
||BsmtCond|HeatingQC|Electrical|BsmtHalfBath|TotRmsAbvGrd|KitchenQual|GarageYrBlt|PoolArea|
||BsmtExposure|CentralAir||FullBath|||GarageFinish|	PoolQC|
||BsmtFinType1|Fireplaces||HalfBath|||GarageCars|Fence|
||BsmtFinSF1|FireplaceQu|||||GarageArea|MiscFeature|
||BsmtFinType2||||||GarageQual|MiscVal|
||BsmtFinSF2||||||GarageCond||
||BsmtUnfSF||||||PavedDrive||

Esta es toda la data separada por categorías para estudiarla en conjuntos.

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib as mpl
import matplotlib.pyplot as plt 
import seaborn as sns 

#### Alrededores y forma.

Donde se encuentran las variables **Street**, **Alley**, **LotShape**, **landContour**, **LotConfig** y **LandSlope**

**Street** 
La variable habla del tipo de acceso a la vivienda. 

Grvl -	Gravel  
Pave -	Paved